## 파이토치를 사용해서 단층 퍼셉트론과 다층 퍼셉트론을 각각 구현하여 XOR 문제를 풀어보자.

### 1. 파이토치로 단층 퍼셉트론 구현하기

In [1]:
import torch
import torch.nn as nn

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)

if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
torch.cuda.is_available()

True

#### 1) 단층 퍼셉트론을 이용한 XOR 문제 풀기

XOR 문제의 입력과 출력을 정의한다.

In [4]:
X = torch.FloatTensor([[0,0],[0,1],[1,0],[1,1]]).to(device)
Y = torch.FloatTensor([[0],[1],[1],[0]]).to(device)

단층이므로 활성 함수로 sigmoid를 사용해도 문제는 없겠다.

In [ ]:
linear = nn.Linear(2,1, bias=True)
sigmoid = nn.Sigmoid()
model = nn.Sequential(linear, sigmoid).to(device)

0 또는 1을 예측하는 이진 분류 문제이므로 비용 함수로는 크로스엔트로피 함수를 사용한다.  
nn.BCELoss()는 이진 분류에서 사용하는 크로스 엔트로피 함수이다.

In [6]:
# 비용 함수와 옵티마이저 정의
criterion = torch.nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1)

In [8]:
for step in range(10001):
    optimizer.zero_grad()
    hypothesis = model(X)

    # 비용 함수
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    if step % 100 == 0: # 100번째 에포크마다 비용 출력
        print(step, cost.item())

0 0.6931471824645996
100 0.6931471824645996
200 0.6931471824645996
300 0.6931471824645996
400 0.6931471824645996
500 0.6931471824645996
600 0.6931471824645996
700 0.6931471824645996
800 0.6931471824645996
900 0.6931471824645996
1000 0.6931471824645996
1100 0.6931471824645996
1200 0.6931471824645996
1300 0.6931471824645996
1400 0.6931471824645996
1500 0.6931471824645996
1600 0.6931471824645996
1700 0.6931471824645996
1800 0.6931471824645996
1900 0.6931471824645996
2000 0.6931471824645996
2100 0.6931471824645996
2200 0.6931471824645996
2300 0.6931471824645996
2400 0.6931471824645996
2500 0.6931471824645996
2600 0.6931471824645996
2700 0.6931471824645996
2800 0.6931471824645996
2900 0.6931471824645996
3000 0.6931471824645996
3100 0.6931471824645996
3200 0.6931471824645996
3300 0.6931471824645996
3400 0.6931471824645996
3500 0.6931471824645996
3600 0.6931471824645996
3700 0.6931471824645996
3800 0.6931471824645996
3900 0.6931471824645996
4000 0.6931471824645996
4100 0.6931471824645996
4200

200에서 epoch 비용이 0.6931...가 출력된 이후 더 이상 비용이 줄어드지 않고 있다.  
단층 퍼셉트론은 XOR 문제를 풀 수 없기 때문이다.  

#### 2) 학습된 단층 퍼셉트론의 예측값 확인하기

In [ ]:
with torch.no_grad():
    hypothesis = model(X)
    # 모델이 예측한 클래스 레이블, 0.5가 넘으면 1, 그렇지 않으면 0으로 간주한다.
    predicted = (hypothesis > 0.5).float()
    # 모델의 예측한 예측값과 실제값을 비교하고, 일치되는 비율을 계산하여 정확도를 구한다.
    accuracy = (predicted == Y).float().mean()

    # .detach().cpu().numpy()로 CPU로 옮겨서 계산한다.
    print(f'모델의 출력값(Hypothesis): {hypothesis.detach().cpu().numpy()}')
    print(f'모델의 예측값(Predicted): {predicted.detach().cpu().numpy()}')
    print(f'실제값(Y): {Y.cpu().numpy()}')
    print(f'정확도(Accuracy): {accuracy.item()}')

모델의 출력값(Hypothesis): [[0.5]
 [0.5]
 [0.5]
 [0.5]]
모델의 예측값(Predicted): [[0.]
 [0.]
 [0.]
 [0.]]
실제값(Y): [[0.]
 [1.]
 [1.]
 [0.]]
정확도(Accuracy): 0.5


### 2. 파이토치로 다층 퍼셉트론 구현하기

In [10]:
import torch
import torch.nn as nn

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

#### 1) 다층 퍼셉트론을 이용한 XOR 문제 풀기

XOR 문제를 풀기 위한 입력과 출력 정의

In [12]:
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
Y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)

다층 퍼셉트론 설계  
입력층, 은닉층1, 은닉층2, 은닉층3, 출력층을 갖는 은닉층 3개의 인공 신경망 설계

In [13]:
model = nn.Sequential(
    nn.Linear(2, 10, bias=True), # input_layer = 2, hidden_layer1 = 10
    nn.Sigmoid(),

    nn.Linear(10,10, bias=True), # hidden_layer1 = 10, hidden_layer2 = 10
    nn.Sigmoid(),

    nn.Linear(10,10,bias=True), # hidden_layer2 = 10, hidden_layer3 = 10
    nn.Sigmoid(),

    nn.Linear(10,1,bias=True), # hidden_layer3 = 10, output_layer = 1
    nn.Sigmoid()
).to(device)

비용 함수와 옵티마이저를 선언.  
nn.BCELoss()는 이진 분류에서 사용하는 크로스엔트로피 함수이다.

In [14]:
criterion = torch.nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1)

총 10,001번의 에포크 수행.  
각 에포크마다 역전파가 수행된다.

In [15]:
for epoch in range(10001):
    optimizer.zero_grad()
    # forward 연산
    hypothesis = model(X)

    # 비용 함수
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    # 100의 배수에 해당되는 에포크마다 비용을 출력
    if epoch % 100 == 0:
        print(epoch, cost.item())

0 0.6948983669281006
100 0.693155825138092
200 0.6931535005569458
300 0.6931514143943787
400 0.6931492686271667
500 0.6931473016738892
600 0.6931453943252563
700 0.6931434869766235
800 0.6931416392326355
900 0.6931397914886475
1000 0.6931380033493042
1100 0.6931361556053162
1200 0.6931343078613281
1300 0.6931324005126953
1400 0.6931304931640625
1500 0.6931284666061401
1600 0.6931264400482178
1700 0.6931242942810059
1800 0.6931220293045044
1900 0.6931196451187134
2000 0.6931171417236328
2100 0.6931145191192627
2200 0.6931116580963135
2300 0.6931085586547852
2400 0.693105161190033
2500 0.6931014657020569
2600 0.6930974721908569
2700 0.6930930614471436
2800 0.6930880546569824
2900 0.6930825710296631
3000 0.6930763721466064
3100 0.6930692791938782
3200 0.6930612325668335
3300 0.6930519342422485
3400 0.6930411458015442
3500 0.693028450012207
3600 0.6930133104324341
3700 0.6929951906204224
3800 0.6929729580879211
3900 0.6929453015327454
4000 0.6929103136062622
4100 0.6928648948669434
4200 0.

#### 2) 학습된 다층 퍼셉트론의 예측값 확인하기

이제 모델이 XOR 문제를 풀 수 있는지 테스트 해보자.

In [16]:
with torch.no_grad():
    hypothesis = model(X)
    predicted = (hypothesis > 0.5).float()
    accuracy = (predicted == Y).float().mean()

    print(f'모델의 출력값(Hypothesis): {hypothesis.detach().cpu().numpy()}' )
    print(f'모델의 예측값(Predicted): {predicted.detach().cpu().numpy()}')
    print(f'실제값(Y): {Y.cpu().numpy()}')
    print(f'정확도(Accuracy): {accuracy.item()}')
    

모델의 출력값(Hypothesis): [[1.1169876e-04]
 [9.9982882e-01]
 [9.9984229e-01]
 [1.8530559e-04]]
모델의 예측값(Predicted): [[0.]
 [1.]
 [1.]
 [0.]]
실제값(Y): [[0.]
 [1.]
 [1.]
 [0.]]
정확도(Accuracy): 1.0
